In [ ]:
import numpy as np
import sklearn.gaussian_process as gp

from scipy.stats import norm
from scipy.optimize import minimize

def improvement(x, gaussian_process, evaluated_loss, greater_is_better=False, n_params=1):

    x_to_predict = x.reshape(-1, n_params)

    mu, sigma = gaussian_process.predict(x_to_predict, return_std=True)

    if greater_is_better:
        loss_optimum = np.max(evaluated_loss)
    else:
        loss_optimum = np.min(evaluated_loss)

    scaling_factor = (-1) ** (not greater_is_better)

    # In case sigma equals zero
    with np.errstate(divide='ignore'):
        Z = scaling_factor * (mu - loss_optimum) / sigma
        improvement = scaling_factor * (mu - loss_optimum) * norm.cdf(Z) + sigma * norm.pdf(Z)
        improvement[sigma == 0.0] == 0.0

    return -1 * improvement


def hyperparameter(acquisition_func, gaussian_process, evaluated_loss, greater_is_better=False,
                               bounds=(0, 10), n_restarts=25):

    best_x = None
    best_acquisition_value = 1
    n_params = bounds.shape[0]

    for starting_point in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, n_params)):

        res = minimize(fun=acquisition_func,
                       x0=starting_point.reshape(1, -1),
                       bounds=bounds,
                       method='L-BFGS-B',
                       args=(gaussian_process, evaluated_loss, greater_is_better, n_params))

        if res.fun < best_acquisition_value:
            best_acquisition_value = res.fun
            best_x = res.x

    return best_x


def bayesian_optimisation(n_iters, sample_loss, bounds, x0=None, n_pre_samples=5,
                          gp_params=None, random_search=False, alpha=1e-5, epsilon=1e-7):

    x_list = []
    y_list = []

    n_params = bounds.shape[0]

    if x0 is None:
        for params in np.random.uniform(bounds[:, 0], bounds[:, 1], (n_pre_samples, bounds.shape[0])):
            x_list.append(params)
            y_list.append(sample_loss(params))
            # print()
    else:
        for params in x0:
            x_list.append(params)
            y_list.append(sample_loss(params))

    xp = np.array(x_list)
    yp = np.array(y_list)

    # Create the GP
    if gp_params is not None:
        model = gp.GaussianProcessRegressor(**gp_params)
    else:
        kernel = gp.kernels.Matern()
        model = gp.GaussianProcessRegressor(kernel=kernel,
                                            alpha=alpha,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)

    for n in range(n_iters):

        model.fit(xp, yp)

        # Sample next hyperparameter
        if random_search:
            x_random = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(random_search, n_params))
            ei = -1 * improvement(x_random, model, yp, greater_is_better=True, n_params=n_params)
            next_sample = x_random[np.argmax(ei), :]
        else:
            next_sample = hyperparameter(improvement, model, yp, greater_is_better=True, bounds=bounds, n_restarts=100)

        # Duplicates will break the GP. In case of a duplicate, we will randomly sample a next query point.
        if np.any(np.abs(next_sample - xp) <= epsilon):
            next_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], bounds.shape[0])

        # Sample loss for new set of parameters
        cv_score = sample_loss(next_sample)

        # Update lists
        x_list.append(next_sample)
        y_list.append(cv_score)

        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)

    return xp, yp

def sample_loss(params):

    return (4 - 2.1 * params[0] ** 2 + params[0] ** 4 / 3) * params[0] ** 2 + params[0] * params[1] + (-4 + 4 * params[1] ** 2) * params[1] ** 2

bounds = np.array([[-3, 3], [-2, 2]])

xp, yp = bayesian_optimisation(n_iters=30,
                               sample_loss=sample_loss,
                               bounds=bounds,
                               n_pre_samples=20,
                               random_search=100000)

print('minimum objective function:', np.min(yp))
print('x1, x2 when having minumum objective function:',xp[np.argmin(yp)])
print('solution x1, x2:', xp)
print('objective function:', yp)

minimum objective function: -0.9293009314259595
x1, x2 when having minumum objective function: [-0.24991534  0.69172499]
solution x1, x2: [[ 0.45766224 -1.2007455 ]
 [-2.29899637 -1.0118714 ]
 [-2.60304181 -0.59318565]
 [-0.56685031 -1.20127258]
 [-1.19064565 -0.75275637]
 [ 0.11249256 -1.78372093]
 [-2.82738315  1.99018875]
 [-1.334834   -1.12495456]
 [ 2.04542842 -0.54624396]
 [ 1.85816847  0.92140323]
 [-2.38817972 -0.49028772]
 [ 2.93281231  0.37968543]
 [-1.66987967 -0.69303279]
 [-2.87925922 -0.5170356 ]
 [-0.75984935  1.92147004]
 [ 1.60846964 -1.12200857]
 [ 0.55573341 -1.74414876]
 [-2.75476971  0.48295992]
 [-0.24991534  0.69172499]
 [ 1.09139514 -0.92622979]
 [-2.99893894  1.83440453]
 [-2.99871842  1.5821012 ]
 [-2.99925645  1.94245581]
 [-2.99173228  1.9788082 ]
 [-2.99545724  1.98357872]
 [ 2.99789075  1.99496589]
 [ 2.99826932  1.62972277]
 [ 2.7832719   1.99971627]
 [ 2.99788634 -1.98192986]
 [ 2.99511813 -1.511444  ]
 [ 2.53395185 -1.99993587]
 [-2.98702191 -1.98958832